In [1]:
import mitsuba as mi
import drjit as dr
import matplotlib.pyplot as plt
import numpy as np
import os
import random
mi.set_variant("cuda_ad_rgb")

## 获取训练数据
每个场景选取2000个相机，每个相机生成一对数据，原论文共包含92个复杂场景
训练数据是成对的，且场景仅包含漫反射  
每一对数据包括两个相邻帧，相邻帧通过稍微改变相机位置或者相机朝向生成。
每一帧包含：
> 1.直接光照图(512X384,2048-4096spps)  
> 2.G-buffers(法线，相机空间3D坐标，albedos)  
> 3.实际的一次弹跳光线间接光图

In [2]:
# scene = mi.load_dict(mi.cornell_box())
scene = mi.load_file("./complex_scenes/living-room/living-room/scene_v3.xml")
aov_integrator = mi.load_dict({
    'type':'aov',
    'aovs':'position:position,normal:sh_normal,depth:depth,albedo:albedo',
    'my_image':{
        'type':'path',
        'max_depth':2
    }
})

In [3]:
def load_camera(camera_transform,spp=4096,width=512,height=384):
    cameras = []
    camera_transform_matrix = camera_transform.matrix.numpy()
    for matrix in camera_transform_matrix:
        cameras.append(mi.load_dict({
            'type':'perspective',
            'fov':39.3077,
            'to_world':mi.ScalarTransform4f(matrix),
            'sampler':{
                'type':'independent',
                'sample_count':spp,
            },
            'film': {
                'type': 'hdrfilm',
                'width': width,
                'height': height,
                'rfilter': {
                    'type': 'tent',
                },
                'pixel_format': 'rgb',
            },
        },True))
    return cameras

def create_cameras(scene,camera_sampler,num_cameras = 2000,seed = 10):
    camera_sampler = camera_sampler.clone()
    camera_sampler.seed(seed,num_cameras)

    # 在场景中随机采样相机位置
    camera_position = scene.bbox().min + scene.bbox().extents()*0.5 + 0.5*mi.warp.square_to_uniform_sphere(camera_sampler.next_2d())
    # 随机采样 视方向
    view_direction =  mi.Point3f(scene.bbox().min + (scene.bbox().max-scene.bbox().min)*mi.Point3f(camera_sampler.next_1d(),camera_sampler.next_1d(),camera_sampler.next_1d()))
    # 确定向上方向
    up = mi.warp.square_to_uniform_hemisphere(camera_sampler.next_2d())
    camera_transform = mi.Transform4f.look_at(camera_position,view_direction,up)
    cameras = load_camera(camera_transform)
    # 添加随机扰动
    if random.uniform(0,1)<0.5:
        camera_position += 0.4*mi.warp.square_to_uniform_sphere(camera_sampler.next_2d())
    else:
        view_direction += 0.4*mi.warp.square_to_uniform_sphere(camera_sampler.next_2d())
    camera_transform_changed = mi.Transform4f.look_at(camera_position,view_direction,up)
    cameras_changed = load_camera(camera_transform_changed)
    return cameras,cameras_changed
    

In [4]:
camera_sampler = mi.load_dict({
    'type':'independent',
    'sample_count':1
})
groundTruth_integrator = mi.load_dict({
    'type':'path',
    'max_depth':3
})
camera_count = 500
spp = 4096
output_dir = "./data/dataSet"
cameras,cameras_changed = create_cameras(scene,camera_sampler,camera_count,seed=12)

In [5]:
import gc
import torch
import os
from tqdm import tqdm 

images = []
for i in tqdm(range(camera_count)):
    image = np.array(mi.render(scene,sensor=cameras[i],integrator=aov_integrator,spp=spp))
    image_groundtruth = np.array(mi.render(scene,sensor=cameras[i],integrator= groundTruth_integrator,spp=spp))
    image_groundtruth = np.clip(image_groundtruth - image[:,:,:3],0,1)

    np.save(os.path.join(output_dir,f"{i+1038}.npy"),image)
    np.save(os.path.join(output_dir,f"{i+1038}_truth.npy"),image_groundtruth)

    del image
    del image_groundtruth
    torch.cuda.empty_cache()
    gc.collect()


100%|██████████| 500/500 [2:31:32<00:00, 18.19s/it]  
